# Imports e instalaciones

In [ ]:
!pip install squarify
import pandas as pd
import numpy as np
import seaborn as sns
import squarify
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# suprimimos la notacion cientifica en los outputs
pd.options.display.float_format = '{:20,.2f}'.format

labelsDF = pd.read_csv('train_labels.csv')
valuesDF = pd.read_csv('train_values.csv')

# Asignacion de datos a columnas segun enunciado

In [4]:
valuesProcessed = valuesDF
valuesProcessed["land_surface_condition"]               = valuesProcessed["land_surface_condition"].astype("category")
valuesProcessed["foundation_type"]                      = valuesProcessed["foundation_type"].astype("category")
valuesProcessed["roof_type"]                            = valuesProcessed["roof_type"].astype("category")
valuesProcessed["ground_floor_type"]                    = valuesProcessed["ground_floor_type"].astype("category")
valuesProcessed["other_floor_type"]                     = valuesProcessed["other_floor_type"].astype("category")
valuesProcessed["position"]                             = valuesProcessed["position"].astype("category")
valuesProcessed["plan_configuration"]                   = valuesProcessed["plan_configuration"].astype("category")
valuesProcessed["legal_ownership_status"]               = valuesProcessed["legal_ownership_status"].astype("category")
valuesProcessed.info()

## Chequeamos de que no hay datos invalidos

Que no tengamos datos nulos y que los datos que tienen un rango de valores en el enunciado, respeten ese rango.

In [9]:
result = valuesProcessed['geo_level_1_id'].loc[(valuesProcessed['geo_level_1_id'] <= 30) & (valuesProcessed['geo_level_1_id'] >= 0)]
result.count()

260601

In [10]:
result = valuesProcessed['geo_level_2_id'].loc[(valuesProcessed['geo_level_2_id'] <= 1427) & (valuesProcessed['geo_level_2_id'] >= 0)]
result.count()

260601

In [11]:
result = valuesProcessed['geo_level_3_id'].loc[(valuesProcessed['geo_level_3_id'] <= 12567) & (valuesProcessed['geo_level_3_id'] >= 0)]
result.count()

260601

In [12]:
valuesProcessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [14]:
labelsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int64
 1   damage_grade  260601 non-null  int64
dtypes: int64(2)
memory usage: 4.0 MB


In [15]:
valuesProcessed['land_surface_condition'].value_counts()

t    216757
n     35528
o      8316
Name: land_surface_condition, dtype: int64

In [16]:
valuesProcessed['foundation_type'].value_counts()

r    219196
w     15118
u     14260
i     10579
h      1448
Name: foundation_type, dtype: int64

In [17]:
valuesProcessed['roof_type'].value_counts()

n    182842
q     61576
x     16183
Name: roof_type, dtype: int64

In [18]:
valuesProcessed['ground_floor_type'].value_counts()

f    209619
x     24877
v     24593
z      1004
m       508
Name: ground_floor_type, dtype: int64

In [19]:
valuesProcessed['other_floor_type'].value_counts()

q    165282
x     43448
j     39843
s     12028
Name: other_floor_type, dtype: int64

In [20]:
valuesProcessed['position'].value_counts()

s    202090
t     42896
j     13282
o      2333
Name: position, dtype: int64

In [21]:
valuesProcessed['plan_configuration'].value_counts()

d    250072
q      5692
u      3649
s       346
c       325
a       252
o       159
m        46
n        38
f        22
Name: plan_configuration, dtype: int64

In [23]:
valuesProcessed['legal_ownership_status'].value_counts()

v    250939
a      5512
w      2677
r      1473
Name: legal_ownership_status, dtype: int64

# Analisis exploratorio

## Uso de edificios

Vamos a analizar, separando por los distintos usos que tiene un edificio, la cantidad de edificios que sufrieron daño. Primero comenzaremos separando por los que tienen uso secundario y los que no.

In [26]:
usoDeEdificios = valuesProcessed.loc[:,['building_id', 'has_secondary_use']]

#Hago un merge left por si hay algun edificio que no haya sufrido daño alguno
usoDeEdificiosMerged = usoDeEdificios.merge(labelsDF, how='left', on='building_id')

# Saco el building_id porque para este caso ya no me sirve.
dfShort = usoDeEdificiosMerged.loc[:,['has_secondary_use','damage_grade']]
dfShort['house'] = dfShort.loc[:,['has_secondary_use']].any(1)
dfShort['house'].replace({False: 1, True: 0}, inplace=True)
dfShort = dfShort.loc[:,['house', 'has_secondary_use', 'damage_grade']]
dfShort.columns = ['Vivienda', 'Uso secundario', 'damage']
dfShort['use'] = dfShort.apply(lambda x: x[x == 1].index[0], axis=1)
dfShort = dfShort.loc[:,['use', 'damage']]

damageDF = dfShort.set_index('use')
damageDFPivot = damageDF.pivot_table(index='use', columns=['damage'], aggfunc='size')
damageDFPivot.columns = ['Daños leves','Daños medios','Daños graves']
damageDFPivot.to_csv('danio_por_uso.csv')

### Uso secundario de edificios

Como hay mucha diferencia en la cantidad de edificios que son viviendas respecto de la cantidad que tiene cada uso secundario. Hacemos un análisis análogo para cada tipo de uso secundario.

In [28]:
usoSecundarioDeEdificios = valuesProcessed.loc[valuesProcessed['has_secondary_use'] == 1,['building_id', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'has_secondary_use_use_police', 'has_secondary_use_other']]
# emprolijo los nombres de las columnas de uso secundario.
def renameSecondaryCols(cols):
    colsList = []
    for col in cols:
        if col == cols[0]:
            colsList.append(col)
            continue
        else: 
            colsList.append(col.split('_')[len(col.split('_')) - 1])
    return colsList
usoSecundarioDeEdificios.columns = renameSecondaryCols(usoSecundarioDeEdificios.columns)
#Hago un merge left por si hay algun edificio que no haya sufrido daño alguno
usoSecundarioMerged = usoSecundarioDeEdificios.merge(labelsDF, how='left', on='building_id')
usoSecundarioMerged = usoSecundarioMerged.drop(columns=['building_id'])
usoSecundarioMerged['use'] = usoSecundarioMerged.apply(lambda x: x[x == 1].index[0], axis=1)

dfSecShort = usoSecundarioMerged.loc[:,['use', 'damage_grade']]
damageSecDF = dfSecShort.set_index('use')
damageSecDFPivot = damageSecDF.pivot_table(index='use', columns=['damage_grade'], aggfunc='size')
damageSecDFPivot.columns = ['Daños leves','Daños medios','Daños graves']
damageSecDFPivot.to_csv('danio_por_uso_sec.csv')

#### Agregamos familias

Ahora hacemos un análisis similar al anterior pero teniendo en cuenta la cantidad de familias que vivían en cada edificio.

In [29]:
familiasYuso = valuesProcessed.loc[:,['building_id', 'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'has_secondary_use_use_police', 'has_secondary_use_other', 'count_families']]

In [30]:
familiasYuso.loc[(familiasYuso['count_families'] != 0) & (familiasYuso['has_secondary_use'] != 0)].size

316901

In [31]:
familiasYuso.loc[(familiasYuso['count_families'] == 0) & (familiasYuso['has_secondary_use'] == 0)].size

209079

Podemos ver que en nuestros datos tenemos tanto edificios de uso secundario que tenían familias viviendo en ellos, como viviendas en las cuales no habia nadie viviendo allí (posiblemente abandonadas)

In [33]:
#Hago un merge left por si hay algun edificio que no haya sufrido daño alguno
familiasYusoMerged = familiasYuso.merge(labelsDF, how='left', on='building_id')
# Saco el building_id porque para este caso ya no me sirve.
dfFamShort = familiasYusoMerged.loc[:,['has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'has_secondary_use_use_police', 'has_secondary_use_other', 'count_families', 'damage_grade']]
dfFamShort['house'] = dfFamShort.loc[:,['has_secondary_use']].any(1)
dfFamShort['house'].replace({False: 1, True: 0}, inplace=True)
dfFamShort = dfFamShort.loc[:,['house', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'has_secondary_use_use_police', 'has_secondary_use_other', 'count_families', 'damage_grade']]
# Emprolijo los nombres de las columnas
def renameFamilieCols(cols):
    colsList = []
    for col in cols:
        if col == 'house':
            colsList.append('vivienda')
            continue
        elif col == 'damage_grade':
            colsList.append('damage')
        else: 
            colsList.append(col.split('_')[len(col.split('_')) - 1])
    return colsList
dfFamShort.columns = renameFamilieCols(dfFamShort.columns)
dfFamShort['use'] = dfFamShort.apply(lambda x: x[x == 1].index[0], axis=1)
famDamageDF = dfFamShort.loc[:,['use','families', 'damage']]
famDamageDF.loc[:,['use','families','damage']].groupby(['use','damage']).sum().to_csv('families_damage.csv')

## Cantidad de familias

Analizamos si la cantidad de familias que viven en las edificaciones influye de alguna manera en el grado de daño que sufrieron.

In [ ]:
joinedDF = valuesProcessed.merge(labelsDF, on='building_id')
joinedDF.value_counts('count_families')
familiesDF = joinedDF.loc[join['count_families'] < 6,['count_families','damage_grade']].groupby('count_families').mean()
familiesDF.to_csv('count_families_damage.csv')

## Edificación (diseño sísimico y materiales)

Analizamos el promedio de daño en las edificaciones, separándolas por una combinación entre el diseño sísimico y los materiales utilizados en su construcción.

In [36]:
edificacion = valuesProcessed.loc[:,['building_id','plan_configuration', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other']]
# Emprolijo los nombres de las columnas
def renameEdificacionCols(cols):
    colsList = []
    for col in cols:
        col.replace('has_superstructure_', '')
        colsList.append(col.replace('has_superstructure_', ''))
    return colsList
edificacion.columns = renameEdificacionCols(edificacion.columns)

#Hago merge left para quedarme con los que esten intactos.
materialDamage = edificacion.merge(labelsDF, how='left', on='building_id')
damagexMat = materialDamage.drop(columns='building_id')

# Me creo otro DF para poder hacer la operación de manera más simple.
# En esta voy a tener la cantidad total de edificios de cada uno.
materialesPorPlan = damagexMat.groupby(['plan_configuration']).sum()
# Ahora voy a tener en cada celda, el promedio de daño.
meanDamage = damagexMat.groupby(['plan_configuration','damage_grade']).sum().fillna(value=0)
meanDamage = meanDamage.reset_index().set_index('plan_configuration')
meanDamage['adobe_mud']           = (meanDamage['damage_grade'] * meanDamage['adobe_mud']) / materialesPorPlan['adobe_mud']
meanDamage['mud_mortar_stone']    = (meanDamage['damage_grade'] * meanDamage['mud_mortar_stone']) / materialesPorPlan['mud_mortar_stone']
meanDamage['stone_flag']          = (meanDamage['damage_grade'] * meanDamage['stone_flag']) / materialesPorPlan['stone_flag']
meanDamage['cement_mortar_stone'] = (meanDamage['damage_grade'] * meanDamage['cement_mortar_stone']) / materialesPorPlan['cement_mortar_stone']
meanDamage['mud_mortar_brick']    = (meanDamage['damage_grade'] * meanDamage['mud_mortar_brick']) / materialesPorPlan['mud_mortar_brick']
meanDamage['cement_mortar_brick'] = (meanDamage['damage_grade'] * meanDamage['cement_mortar_brick']) / materialesPorPlan['cement_mortar_brick']
meanDamage['timber']              = (meanDamage['damage_grade'] * meanDamage['timber']) / materialesPorPlan['timber']
meanDamage['bamboo']              = (meanDamage['damage_grade'] * meanDamage['bamboo']) / materialesPorPlan['bamboo']
meanDamage['rc_non_engineered']   = (meanDamage['damage_grade'] * meanDamage['rc_non_engineered']) / materialesPorPlan['rc_non_engineered']
meanDamage['rc_engineered']       = (meanDamage['damage_grade'] * meanDamage['rc_engineered']) / materialesPorPlan['rc_engineered']
meanDamage['other']               = (meanDamage['damage_grade'] * meanDamage['other']) / materialesPorPlan['other']
meanDamage = meanDamage.reset_index().groupby('plan_configuration').sum().drop(columns=['damage_grade'])
meanDamage.columns = ['Adobe/Barro','Barro-Piedra','Piedra','Cemento-Piedra','Barro-Ladrillos','Cemento-Ladrillos','Madera','Bambú','Concreto reforzado no diseñado','Concreto reforzado diseñado','Otro']
meanDamage.to_csv('mean_damage.csv')

## Ubicación greográfica

Tomando algunas consideraciones que serán mencionadas en el informe, vamos a analizar el daño por región para poder concluir cuál fue el epicentro del terremoto.

In [ ]:
geoLoc = valuesProcessed.loc[:,['building_id','geo_level_1_id','geo_level_2_id','geo_level_3_id']]
geoLoc['geo_id'] = geoLoc['geo_level_1_id'].astype(str).str.zfill(2) + '-' + geoLoc['geo_level_2_id'].astype(str).str.zfill(4) + '-' + geoLoc['geo_level_3_id'].astype(str).str.zfill(5)

geoLocToMerge = geoLoc.loc[:,['building_id','geo_id']]
#Hago un merge left por si hay algun edificio que no haya sufrido daño alguno
geoDamage = geoLocToMerge.merge(labelsDF, how='left', on='building_id').loc[:,['geo_id','damage_grade']]
geoDamage.groupby(['geo_id']).agg({'damage_grade':'mean'}).to_csv('geo_damage.csv')

Ahora haremos un análisis más dedicado a las regiones

### Análisis de las distintas regiones

In [39]:
analisisDeRegiones = valuesProcessed.loc[:,['building_id','geo_level_1_id','geo_level_2_id','geo_level_3_id','age','area_percentage','height_percentage','count_families','count_floors_pre_eq']]
analisisDeRegiones = analisisDeRegiones.merge(labelsDF)

Vamos a analizar los dos extremos de daños (grado 1 y 3) para cada nivel de región.

#### Promedios por region lvl1

In [44]:
promediosPorRegionlvl1 = analisisDeRegiones.groupby('geo_level_1_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionlvl1.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

promediosPorRegionLvl1YDamageGrade3 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 3,:].groupby('geo_level_1_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionLvl1YDamageGrade3.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

promediosPorRegionLvl1YDamageGrade1 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 1,:].groupby('geo_level_1_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionLvl1YDamageGrade1.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

#### Promedios por region lvl2

In [45]:
promediosPorRegionlvl2 = analisisDeRegiones.groupby('geo_level_2_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionlvl2.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

promediosPorRegionLvl2YDamageGrade3 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 3,:].groupby('geo_level_2_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionLvl2YDamageGrade3.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

promediosPorRegionLvl2YDamageGrade1 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 1,:].groupby('geo_level_2_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionLvl2YDamageGrade1.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

#### Promedios por region lvl3

In [47]:
promediosPorRegionlvl3 = analisisDeRegiones.groupby('geo_level_3_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionlvl3.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

promediosPorRegionLvl3YDamageGrade3 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 3,:].groupby('geo_level_3_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionLvl3YDamageGrade3.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

promediosPorRegionLvl3YDamageGrade1 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 1,:].groupby('geo_level_3_id').agg({'age':'mean','area_percentage':'mean','height_percentage':'mean','count_families':'mean','count_floors_pre_eq':'mean'})
promediosPorRegionLvl3YDamageGrade1.columns = ['age_mean','area_percentage_mean','height_percentage_mean','count_families_mean','count_floors_pre_eq_mean']

#### Top 10 regiones de lvl1 con daño de grado = 3 y grado = 1

In [49]:
regioneslvl1ConDamageGrade3 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 3,:]
regioneslvl1ConDamageGrade3 = regioneslvl1ConDamageGrade3.groupby('geo_level_1_id').agg({'damage_grade':'count'}).nlargest(10,'damage_grade')
regioneslvl1ConDamageGrade3.columns = ['cant_damage_grade_3']

regioneslvl1ConDamageGrade1 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 1,:]
regioneslvl1ConDamageGrade1 = regioneslvl1ConDamageGrade1.groupby('geo_level_1_id').agg({'damage_grade':'count'}).nlargest(10,'damage_grade')
regioneslvl1ConDamageGrade1.columns = ['cant_damage_grade_1']

#### Top 10 regiones de lvl2 con daño de grado = 3 y grado = 1

In [50]:
regioneslvl2ConDamageGrade3 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 3,:]
regioneslvl2ConDamageGrade3 = regioneslvl2ConDamageGrade3.groupby('geo_level_2_id').agg({'damage_grade':'count'}).nlargest(10,'damage_grade')
regioneslvl2ConDamageGrade3.columns = ['cant_damage_grade_3']

regioneslvl2ConDamageGrade1 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 1,:]
regioneslvl2ConDamageGrade1 = regioneslvl2ConDamageGrade1.groupby('geo_level_2_id').agg({'damage_grade':'count'}).nlargest(10,'damage_grade')
regioneslvl2ConDamageGrade1.columns = ['cant_damage_grade_1']

#### Top 10 regiones de lvl3 con daño de grado = 3 y grado = 1

In [51]:
regioneslvl3ConDamageGrade3 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 3,:]
regioneslvl3ConDamageGrade3 = regioneslvl3ConDamageGrade3.groupby('geo_level_3_id').agg({'damage_grade':'count'}).nlargest(10,'damage_grade')
regioneslvl3ConDamageGrade3.columns = ['cant_damage_grade_3']

regioneslvl3ConDamageGrade1 = analisisDeRegiones.loc[analisisDeRegiones.damage_grade == 1,:]
regioneslvl3ConDamageGrade1 = regioneslvl3ConDamageGrade1.groupby('geo_level_3_id').agg({'damage_grade':'count'}).nlargest(10,'damage_grade')
regioneslvl3ConDamageGrade1.columns = ['cant_damage_grade_1']

##### Estadísiticas para cada region de cada lvl con mayor cantidad de ocurrencias de daño de grado 3

In [52]:
estadisticasRegionesConMasDamageGrade3 = regioneslvl1ConDamageGrade3.nlargest(1,'cant_damage_grade_3').join(promediosPorRegionlvl1).reset_index()
estadisticasRegionesConMasDamageGrade3 = estadisticasRegionesConMasDamageGrade3.append(regioneslvl2ConDamageGrade3.nlargest(1,'cant_damage_grade_3').join(promediosPorRegionlvl2).reset_index())
estadisticasRegionesConMasDamageGrade3 = estadisticasRegionesConMasDamageGrade3.append(regioneslvl3ConDamageGrade3.nlargest(1,'cant_damage_grade_3').join(promediosPorRegionlvl3).reset_index())
estadisticasRegionesConMasDamageGrade3['id'] = ['Geo_level_1 = 17','Geo_level_2 = 363','Geo_level_3 = 8236']

##### Estadísiticas para cada region de cada lvl con mayor cantidad de ocurrencias de daño de grado 1

In [53]:
estadisticasRegionesConMasDamageGrade1 = regioneslvl1ConDamageGrade1.nlargest(1,'cant_damage_grade_1').join(promediosPorRegionlvl1).reset_index()
estadisticasRegionesConMasDamageGrade1 = estadisticasRegionesConMasDamageGrade1.append(regioneslvl2ConDamageGrade1.nlargest(1,'cant_damage_grade_1').join(promediosPorRegionlvl2).reset_index())
estadisticasRegionesConMasDamageGrade1 = estadisticasRegionesConMasDamageGrade1.append(regioneslvl3ConDamageGrade1.nlargest(1,'cant_damage_grade_1').join(promediosPorRegionlvl3).reset_index())
estadisticasRegionesConMasDamageGrade1['id'] = ['Geo_level_1 = 26','Geo_level_2 = 39','Geo_level_3 = 9133']

## Antigüedad de las edificaciones

In [11]:
age = valuesProcessed.loc[:, ['building_id', 'age']]
agedmg = age.merge(labelsDF, on='building_id', how='inner')
agedmg['id'] = 1
agedmg = agedmg.drop(columns=['building_id'])

In [12]:
agedmg = agedmg.set_index('age')
agedmgavg = agedmg.groupby(['age']).agg({'id': 'sum', 'damage_grade':'mean'})
agedmgavg = agedmgavg.rename(columns={'id': 'building_count'})

## Análisis de altura y materiales en función de la destrucción

Primero, queremos ver de qué me sirve la altura y el porcentaje del área. Por lo que queremos ver si la cantidad de pisos y el porcentaje de altura tienen alguna relación.

In [ ]:
labelsDF
valuesDF[["height_percentage", "count_floors_pre_eq"]].head(10)

Podemos ver de entrada que no parece haber mucha relación entre ambos. Tomaremos la cantidad de pisos como indicador real de la altura del edifico. Ahora, veremos el porcentaje de destrucción para las distintas alturas.

In [ ]:
height_only = valuesDF[["building_id", "count_floors_pre_eq"]].merge(labelsDF, on="building_id")
height_only[["count_floors_pre_eq", "damage_grade"]].groupby("count_floors_pre_eq").count()

Tomaremos sólo los que tienen más de 200 valores para el calculo, es decir desde el piso 1 al 6. También es útil ver la cantidad de edificios.

In [ ]:
#height_only_filtered = height_only[height_only["count_floors_pre_eq"] < 7]
damage_per_height = height_only[["count_floors_pre_eq", "damage_grade"]].groupby("count_floors_pre_eq").count().reset_index()
damage_per_height.columns = ["cantidad de pisos", "cantidad de edificios"]
damage_per_height

In [ ]:
damage_per_height.set_index("count_floors_pre_eq").plot.bar(label= "Promedio de daño por altura")
damage_per_height.set_index("count_floors_pre_eq").to_csv("alturas_simple.csv")

Uno creeria que la altura seria un factor importante en la estabilidad de un edificio. Sin embargo, se puede ver en los números que el mayor porcentaje de destrucción es en los edificios de 3 pisos y disminuye en las siguientes alturas. Nuestra teoría es que es posible hacer edificios de pocos pisos de forma precaria, pero es necesario una mayor inversión y por lo tanto una mayor planificación para hacer edificios de mayor altura.

Para nuestro próximo análisis, veremos si para los mismos materiales de construcción hay una variación en el promedio de daño por altura. Ya que si bien como se vió en el análisis anterior la altura no es un factor que pueda permitir predecir el daño por si misma, quizas es posible construir una tabla o factor de destrucción para los materiales en función de la altura. Intentaremos darle un factor a cada uno de los distintos campos (ground_floor_tpye, other_floor_type, superstructure, land_surface_condition) intentaremos luego combinar estos resultado intentando tener en cuenta que tanto influyen en el factor de destrucción y qué importancia tienen.

In [ ]:
height_ground_floor = valuesDF[["building_id", "count_floors_pre_eq", "ground_floor_type"]].merge(labelsDF, on="building_id")
height_ground_floor = height_ground_floor.set_index("building_id")
height_ground_floor = height_ground_floor[height_ground_floor["count_floors_pre_eq"] < 7]
ground_floor_type_count =  height_ground_floor.groupby(["ground_floor_type", "count_floors_pre_eq"]).agg({"damage_grade": ["count", "mean", "std"]})
ground_floor_type_count

Posibles gráficos: utilización para cada material en edificios de distintos pisos. Por material, el nivel de destrucción por altura, atención al std porque indica qué tanto varía el resultado de la destrucción, indicaría en un principio qué valor es el más influyente.

Conclusiones, al menos en este campo se respeta una estructura similar al de las alturas comunes. Se tiene un mayor porcentaje de daño.

In [ ]:
usefull_attributes = ["land_surface_condition", "foundation_type", "ground_floor_type", "other_floor_type"]
results_height = {}
for elem in usefull_attributes:
    height_foundation_type  = valuesDF[["building_id", "count_floors_pre_eq", elem]].merge(labelsDF, on="building_id")
    height_foundation_type = height_foundation_type.set_index("building_id")
    height_foundation_type = height_foundation_type[height_foundation_type["count_floors_pre_eq"] < 7]
    height_foundation_type =  height_foundation_type.groupby([elem, "count_floors_pre_eq"]).agg({"damage_grade": ["count", "mean", "std"]})
   # height_foundation_type =  height_foundation_type.groupby([elem, "count_floors_pre_eq"]).agg({"damage_grade": "mean"}).reset_index()
    height_foundation_type.columns = [elem, "Pisos", "Daño promedio"]
    #height_foundation_type = height_foundation_type.pivot_table(height_foundation_type, index =elem, columns="Pisos")
    results_height[elem] = height_foundation_type

results_height["land_surface_condition"]

In [ ]:
for key in results_height:
    results_height[key].to_csv("graficosLopez/" + key + "_height" + ".csv")

In [ ]:
usefull_attributes = ["land_surface_condition", "foundation_type", "ground_floor_type", "other_floor_type"]
results = {}
for elem in usefull_attributes:
    foundation_type  = valuesDF[["building_id", elem]].merge(labelsDF, on="building_id")
    foundation_type = foundation_type.set_index("building_id")
    foundation_type =  foundation_type.groupby(elem).agg({"damage_grade": ["count", "mean", "std"]})
    results[elem] = foundation_type

In [ ]:
results["land_surface_condition"]
for key in results:
    results[key].to_csv("graficosLopez/" + key + ".csv")